# Decision tree

## Imports

In [ ]:
# Imports
import sys

# Adds the utils directory to the PYTHONPATH
sys.path.append("../utils")  

# Imports everything from common_imports.py
from common_imports import *  

# Adds the project root to the PYTHONPATH
sys.path.append("..")  

# Load train / test datasets
# Train sets
X_train = pd.read_csv(X_train_path)
y_train = pd.read_csv(y_train_path)

# Test sets
X_test = pd.read_csv(X_test_path)
y_test = pd.read_csv(y_test_path)

In [ ]:
# Instantiate a decision tree model
tree = DecisionTreeClassifier(random_state=42)

# Dictionnary of hyperparameters
cv_params = {'max_depth' : [4, 6, 8, None],
             'min_samples_leaf' : [1, 2, 5],
             'min_samples_split' : [2, 4, 6]}

# Scoring
scoring = {'f1' : 'f1', 
           'accuracy' : 'accuracy', 
           'precision' : 'precision', 
           'recall' : 'recall', 
           'roc_auc' : 'roc_auc'}

# Instantiate GridSearch
tree_cv1 = GridSearchCV(tree, cv_params, scoring=scoring, cv=5, refit='roc_auc', verbose=True, n_jobs=-1) 


Fit the decision tree model to the training data.

In [ ]:
%%time
tree_cv1.fit(X_train, y_train)

In [ ]:
# Define saving path
path = '../data/results/'

# Write pickle
write_pickle(path, tree_cv1, 'hr_dt1')

In [ ]:
# Best parameters
tree_cv1.best_params_

In [ ]:
# Best score
tree_cv1.best_score_

This is a strong AUC score, which shows that this model can predict employees who will leave very well.

Use utility function to get all the scores from grid search.

In [ ]:
# Load `model_results` DataFrame
results = pd.read_csv(model_results_path)

# Get all CV scores
tree1_cv_results = make_results('decision tree cv', tree_cv1, 'auc')

# Show model results
tree1_cv_results

In [ ]:
# Update `results` DataFrame
results = pd.concat([results, tree1_cv_results], axis=0)

# Sort 'results' by F1-score
results = results.sort_values(by='F1', ascending=False).reset_index(drop=True)

# Compare results to other saved results
results

All of these scores from the decision tree model are strong indicators of good model performance.

In [ ]:
# Get predictions from test data
tree_test_scores = get_scores('decision tree test', tree_cv1, X_test, y_test)

# Update `results` DataFrame
results = pd.concat([results, tree_test_scores], axis=0)

# Sort 'results' by F1-score
results = results.sort_values(by='F1', ascending=False).reset_index(drop=True)

# Save `results` DataFrame
results.to_csv('../model/model_results.csv', index=False)

# Display `results` Dataframe
results

## Tree Visualisation

In [ ]:
# Extract column names
feature_names = list(X.columns)

# Class names
class_names = ['stayed', 'left']

# Plot the tree
plt.figure(figsize=(85,20))
plot_tree(tree_cv1.best_estimator_, 
          max_depth=6, fontsize=14, 
          feature_names=feature_names, 
          class_names=class_names, 
          filled=True);
plt.show()

## Feature Importance

In [ ]:
tree_importances = pd.DataFrame(tree_cv1.best_estimator_.feature_importances_, 
                                 columns=['gini_importance'], 
                                 index=X.columns
                                )
tree_importances = tree_importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
tree_importances = tree_importances[tree_importances['gini_importance'] != 0]
tree_importances

In [ ]:
sns.barplot(data=tree_importances, x="gini_importance", y=tree_importances.index, orient='h')
plt.title("Decision Tree: Feature Importances for Employee Leaving", fontsize=12)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()